In [1]:
import numpy as np
from scipy.integrate import quad
import pygame 

from formula_functions import *
from animation_functions import *
from intersection_functions import create_spiral, distance
from show_parameters_functions import *
from key_controls_functions import *


from variables import *
from Data_classes.data_processing import DataProcessing


pygame 2.4.0 (SDL 2.26.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [6]:
def main_loop(screen_width, screen_height, FPS):

    '''
    Initializations
    
    '''
    
    pygame.init()

    font_small = pygame.font.Font(None, 26)

    bg_color = (255, 255, 255)
    line_color = (185, 185, 185) # Color of the coordinate line
    
    clock = pygame.time.Clock()
    
    # Layers
    win = pygame.display.set_mode((screen_width, screen_height)) # Main animation layer

    
    data_processing = DataProcessing()
    
    
    # Dictionaries
    
    updates_dict = data_processing.booleans.update_booleans_dict
    
    steps_dict_constants = data_processing.constants.steps_constants_dict    

    const_params_dict = data_processing.constants.constants_dict
    
    var_params_dict = data_processing.variables.variables_dict
 
    mode_statuses_dict = data_processing.mode_statuses_dict
  

    algorithm_variables_dict = data_processing.algorithm_vars.algorithm_vars_dict
    
    t_mth_aproxim_list = [] # caches the already computed approximation values
    
    
    # Create background coordinate system
    create_background(mode_statuses_dict['Coordinates'][0], const_params_dict['c'], var_params_dict['c'],
                      screen_width, screen_height, length, bg_color, font_small)

    draw_vertical_line(mode_statuses_dict['Vertical line'][0], const_params_dict['c'], var_params_dict['c'], screen_height, length)
    
    
    draw_derivatives(mode_statuses_dict['Derivatives'][0], const_params_dict['deg'],t, v, w, k, const_params_dict['c'], var_params_dict['c'], 
                     screen_width, screen_height, length, t_diagram=mode_statuses_dict['T-diagram'][1],
                     show_derivatives=mode_statuses_dict['Derivatives'][1])

    draw_spiral(mode_statuses_dict['Spiral'][0], const_params_dict['c'], var_params_dict['c'],half_screen_width, half_screen_height,
                length,deg=deg, t=t, v=v,w=w,k=k, t_diagram_mode=mode_statuses_dict['T-diagram'][1])
    
    t_nth_list = calc_y_intersects_t(const_params_dict['t'], const_params_dict['w'], const_params_dict['k'])
   
        
    # Initial drawing of the y-intersection point
    draw_dots(mode_statuses_dict['Y-intersects'][0], const_params_dict['c'], var_params_dict['c'], length, t_nth_list, deg, v, w, k, 'black')
    
    show_parameters(mode_statuses_dict['Parameters'][0], font_small, const_params_dict,
                    var_params_dict)
    
    show_mode_statuses(mode_statuses_dict['Modes layer'][0], font_small, mode_statuses_dict)
    
    

    data_processing.blit_layers(win)
    
    pygame.display.update()
 
    running = True
    
    while running:
        
        clock.tick(FPS)
        
        updates_dict['update_screen'], updates_dict['update_spiral'],\
        updates_dict['update_line'], updates_dict['reset_background'] = False, False, False, False
      
        # Check and handle for key control events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            else:
    
                updates_dict, mode_statuses_dict = handle_ctrl_commands(event, updates_dict, mode_statuses_dict)
    
                if mode_statuses_dict['Algorithm mode'][1]:

                    algorithm_variables_dict, updates_dict = handle_algorithm_mode_controls(event, algorithm_variables_dict,
                                                                                            t_mth_aproxim_list, updates_dict)
                    
                else:
 
                    updates_dict, const_params_dict, \
                    var_params_dict, mode_statuses_dict = handle_shift_key_commands(event, updates_dict,
                                                                                    const_params_dict, var_params_dict,
                                                                                    mode_statuses_dict)

                if updates_dict['is_turn_off'] is not None:
                    
                    mode_statuses_dict['Algorithm mode'][0].fill((0, 0, 0,0))
                    mode_statuses_dict['Algorithm data mode'][0].fill((0, 0, 0, 0))
                    t_mth_aproxim_list.clear()
                    algorithm_variables_dict = {'n': 0, 'm': 0, 'total_n': 0}
                    mode_statuses_dict['Derivatives'][1] = False

        const_params_dict, var_params_dict,\
        steps_dict_constants, updates_dict,\
        mode_statuses_dict['Algorithm mode'][1] = handle_key_commands(const_params_dict, var_params_dict,
                                                                      steps_dict_constants,updates_dict, 
                                                                      mode_statuses_dict['Algorithm mode'][1])

        if updates_dict['update_screen']:

          
            curr_x = const_params_dict['x'] + var_params_dict['x']
            curr_t = const_params_dict['t'] + var_params_dict['t']
            curr_v = const_params_dict['v'] + var_params_dict['v']
            curr_w = const_params_dict['w'] + var_params_dict['w']
            curr_k = const_params_dict['k'] + var_params_dict['k']
            curr_l = const_params_dict['l'] + var_params_dict['l']
            curr_deg = const_params_dict['deg'] + var_params_dict['deg']
            
            if mode_statuses_dict['Algorithm mode'][1]:
                
                mode_statuses_dict['Derivatives'][1] = False
                
            draw_derivatives(mode_statuses_dict['Derivatives'][0],curr_deg, curr_t, curr_v, curr_w, curr_k,
                                 const_params_dict['c'], var_params_dict['c'], screen_width, screen_height, curr_l,
                                 t_diagram=mode_statuses_dict['T-diagram'][1],
                                 show_derivatives=mode_statuses_dict['Derivatives'][1])
            
            if updates_dict['shift_coords'] or updates_dict['reset_background']:
 
                # Update coordinate system
                create_background(mode_statuses_dict['Coordinates'][0], const_params_dict['c'], var_params_dict['c'],
                                      screen_width, screen_height, curr_l, bg_color, font_small)

            
            if updates_dict['update_spiral'] or updates_dict['is_t_diagram_change']:
                
                # mode_statuses_dict['Vertical line'][1] = False if mode_statuses_dict['T-diagram'][1] else True
                    
                draw_spiral(mode_statuses_dict['Spiral'][0], const_params_dict['c'], var_params_dict['c'], 
                            half_screen_width, half_screen_height, curr_l,
                            deg=curr_deg,
                            t= curr_t, 
                            v= curr_v,
                            w= curr_w,
                            k= curr_k,
                            t_diagram_mode=mode_statuses_dict['T-diagram'][1])
                   
               
                
            if updates_dict['update_line']:         
                
                draw_vertical_line(mode_statuses_dict['Vertical line'][0], const_params_dict['c'], var_params_dict['c'],
                                   screen_height, curr_l, x_axis_value=curr_x)
             

            # A list contains the exact t-parameters for the intersection points of the spiral with y-axis
            t_nth_list = calc_y_intersects_t(curr_t, curr_w, curr_k)
            
            # Draw current y intersection points if y-intersects mode is `on`
            if mode_statuses_dict['Y-intersects'][1]:
             
                draw_dots(mode_statuses_dict['Y-intersects'][0], const_params_dict['c'], var_params_dict['c'], curr_l,
                          t_nth_list, curr_deg, curr_v, curr_w, curr_k, 'black', t_diagram=mode_statuses_dict['T-diagram'][1])

            # A list contains the exact t-parameters for the intersection points of the spiral with the vertical line
            t_mth_list = calc_line_intersections_t(t_nth_list, deg, curr_x, curr_v, curr_w, curr_k, i=400, accuracy=7)
            
            # Draw current line intersection points if line-intersects mode is `on`
            if mode_statuses_dict['Line-intersects'][1]:
                draw_dots(mode_statuses_dict['Line-intersects'][0], const_params_dict['c'], var_params_dict['c'], curr_l, 
                          t_mth_list, curr_deg, curr_v, curr_w, curr_k, 'purple', t_diagram=mode_statuses_dict['T-diagram'][1])

            # Show the current statuses of the spiral and line parameters
            show_parameters(mode_statuses_dict['Parameters'][0], font_small, const_params_dict, var_params_dict)  
            
            # Show the current statuses of the mode parameters
            show_mode_statuses(mode_statuses_dict['Modes layer'][0], font_small, mode_statuses_dict)

            
            
            # Show the algorithm steps towards the interesection point
            if mode_statuses_dict['Algorithm mode'][1]:
                
                '''algorithm_layer, t_nth_list, deg, v, w, k, x, t_mth_aproxim_list, algorithm_variables_dict, 
                        const_center_point, var_center_point, length,
                         accuracy=5, curr_rad_vec_color='black', previous_rad_vec_color='lightgreen'''
           
                t_mth_aproxim_list, \
                algorithm_variables_dict = draw_algorithm_steps(mode_statuses_dict['Algorithm mode'][0], t_nth_list, 
                                                                curr_deg, curr_v, curr_w, curr_k, curr_x, 
                                                                t_mth_aproxim_list,algorithm_variables_dict,
                                                                const_params_dict['c'], var_params_dict['c'], curr_l,
                                                                accuracy= 5)
            
                x, y = 'None', 'None'
                if len(t_mth_aproxim_list) > 1:
         
                    mth_t = t_mth_aproxim_list[algorithm_variables_dict['m']]
            
                    center_point_width = const_params_dict['c'][0] + var_params_dict['c'][0]
                    center_point_height = const_params_dict['c'][1] + var_params_dict['c'][1]
             
                    x, y = calc_single_t_aproxim(curr_deg, curr_v, curr_w, curr_k,  mth_t, center_point_width,
                                                 center_point_height, curr_l, transform=False)
                
                # Show the current statuses of the m_th aproximation point
                show_algorithm_rows_and_cols(mode_statuses_dict['Algorithm data mode'][0], x, y, algorithm_variables_dict, font_small)
                
            # Attache the new drawings on the screen    
            data_processing.blit_layers(win)
            
            pygame.display.update()

        
    # Exit
    pygame.quit()
    
main_loop(screen_width, screen_height, FPS)